In [98]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re
import spacy
from collections import Counter

from sklearn import ensemble
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression

from sklearn.svm import SVC
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

In [2]:
from nltk.corpus import gutenberg, stopwords

In [42]:
# Utility function to clean text.
def text_cleaner(text, pattern1, pattern2):
    
    # Visual inspection shows spaCy does not recognize the double dash '--'.
    # Better get rid of it now!
    text = re.sub(r'--',' ',text)
    
    # Get rid of headings in square brackets.
    text = re.sub("[\[].*?[\]]", "", text)
    
    # Get rid of chapter titles.
    text = re.sub(pattern1,'',text)
    text = re.sub(pattern2,'',text)
    
    # Get rid of extra whitespace.
    text = ' '.join(text.split())
    
    text = text.lower()
    
    return text

In [43]:
caesar = gutenberg.raw('shakespeare-caesar.txt')
hamlet = gutenberg.raw('shakespeare-hamlet.txt')
macbeth = gutenberg.raw('shakespeare-macbeth.txt')

caesar = text_cleaner(caesar, r'Actus .*', r'Scoena .*')
hamlet = text_cleaner(hamlet, r'Actus .*', r'Scoena .*')
macbeth = text_cleaner(macbeth, r'Actus .*', r'Scoena .*')

In [263]:
macbeth

"thunder and lightning. enter three witches. 1. when shall we three meet againe? in thunder, lightning, or in raine? 2. when the hurley-burley's done, when the battaile's lost, and wonne 3. that will be ere the set of sunne 1. where the place? 2. vpon the heath 3. there to meet with macbeth 1. i come, gray-malkin all. padock calls anon: faire is foule, and foule is faire, houer through the fogge and filthie ayre. exeunt. scena secunda. alarum within. enter king malcome, donalbaine, lenox, with attendants, meeting a bleeding captaine. king. what bloody man is that? he can report, as seemeth by his plight, of the reuolt the newest state mal. this is the serieant, who like a good and hardie souldier fought 'gainst my captiuitie: haile braue friend; say to the king, the knowledge of the broyle, as thou didst leaue it cap. doubtfull it stood, as two spent swimmers, that doe cling together, and choake their art: the mercilesse macdonwald (worthie to be a rebell, for to that the multiplying v

In [24]:
nlp = spacy.load('en')

In [45]:
caesar_doc = nlp(caesar)

In [46]:
hamlet_doc = nlp(hamlet)

In [47]:
macbeth_doc = nlp(macbeth)

In [37]:
# Utility function to calculate how frequently words appear in the text.
def word_frequencies(text, include_stop=False):
    
    # Build a list of words.
    # Strip out punctuation and, optionally, stop words.
    words = []
    for token in text:
        if not token.is_punct and (not token.is_stop or include_stop):
            words.append(token.text)
            
    # Build and return a Counter object containing word counts.
    return Counter(words)

In [48]:
caesar_freq = word_frequencies(caesar_doc, include_stop=False).most_common(10)
print('Caesar:', caesar_freq)

Caesar: [('caesar', 189), ('brutus', 161), ('bru', 153), ('haue', 147), ('shall', 125), ('thou', 115), ('cassi', 107), ('cassius', 85), ('antony', 75), ('come', 74)]


In [49]:
hamlet_freq = word_frequencies(hamlet_doc, include_stop=False).most_common(10)
print('Hamlet:', hamlet_freq)

Hamlet: [('ham', 337), ('lord', 211), ('haue', 175), ('king', 172), ("'s", 122), ('shall', 107), ('thou', 104), ('let', 104), ('come', 104), ('hamlet', 99)]


In [50]:
macbeth_freq = word_frequencies(macbeth_doc, include_stop=False).most_common(10)
print('Hamlet:', macbeth_freq)

Hamlet: [('macb', 137), ("'s", 131), ('haue', 122), ('thou', 87), ('enter', 81), ('shall', 68), ('macbeth', 61), ('thee', 61), ('vpon', 60), ('macd', 58)]


In [51]:
# Group into sentences.
caesar_sents = [[sent, "Caesar"] for sent in caesar_doc.sents]
hamlet_sents = [[sent, "Hamlet"] for sent in hamlet_doc.sents]
macbeth_sents = [[sent, "Macbeth"] for sent in macbeth_doc.sents]

In [53]:
# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(caesar_sents + macbeth_sents + macbeth_sents)
sentences.head()

,0,1
0,"(enter, flauius, ,, murellus, ,, and, certaine...",Caesar
1,"(flauius, .)",Caesar
2,"(hence, :, home, you, idle, creatures, ,, get,...",Caesar
3,"(what, ,, know, you, not, (, being, mechanical...",Caesar
4,"(you, ought, not, walke, vpon, a, labouring, d...",Caesar


In [124]:
sentences.shape

(4849, 2)

In [54]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]

In [57]:
bow_caesar = bag_of_words(caesar_doc)
bow_hamlet = bag_of_words(hamlet_doc)
bow_macbeth = bag_of_words(macbeth_doc)

In [58]:
common_words = set(bow_caesar + bow_hamlet + bow_macbeth)

In [60]:
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

In [61]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500


,vnrighteous,taper,cheere,gaming,stab'd,flagon,prophetique,fly,illusion,backward,...,hew,physick,flye,infect,end,rapier,aboord,bellow,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(enter, flauius, ,, murellus, ,, and, certaine...",Caesar
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(flauius, .)",Caesar
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(hence, :, home, you, idle, creatures, ,, get,...",Caesar
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(what, ,, know, you, not, (, being, mechanical...",Caesar
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(you, ought, not, walke, vpon, a, labouring, d...",Caesar


### With Random Forest... the cross validation test scores range in the 0.85-0.88 range.

In [63]:
rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.980749398419

Test set score: 0.864948453608


In [65]:
scores = cross_val_score(rfc, X, Y, cv=5)
scores    

array([ 0.87641607,  0.85979381,  0.87216495,  0.8875129 ,  0.85758514])

### Also in a similiar range for Logistic Regression- this time 0.84-0.88.

In [84]:
lr = LogisticRegression(solver='newton-cg')
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(2909, 4012) (2909,)
Training set score: 0.941216913029

Test set score: 0.867525773196


In [85]:
scores2 = cross_val_score(lr, X, Y, cv=5)
scores2

array([ 0.86096807,  0.84329897,  0.88041237,  0.875129  ,  0.84623323])

### The lowest scores so far for Gradient Boosting Classifier...

In [69]:
clf = ensemble.GradientBoostingClassifier()

train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.830525953936

Test set score: 0.812886597938


In [70]:
scores3 = cross_val_score(clf, X, Y, cv=5)
scores3 

array([ 0.81359423,  0.78041237,  0.83298969,  0.82765738,  0.78637771])

### Support Vector Machine did the best overall...

In [74]:
svm = SVC(kernel = 'linear')
svm.fit(X_train, y_train)

print('Training set score:', svm.score(X_train, y_train))
print('\nTest set score:', svm.score(X_test, y_test))

Training set score: 0.961155036095

Test set score: 0.865979381443


In [75]:
scores4 = cross_val_score(svm, X, Y, cv=5)
scores4 

array([ 0.87435633,  0.86597938,  0.88453608,  0.875129  ,  0.86377709])

### So Support Vector Machine did best overall. 

In [86]:
print("Random Forest: ", np.mean(scores))
print("Logistic Regression: ", np.mean(scores2))
print("Gradient Boosting Classifier: ", np.mean(scores3))
print("Support Vector Machine: ", np.mean(scores4))

Random Forest:  0.870694573603
Logistic Regression:  0.861208328692
Gradient Boosting Classifier:  0.808206276465
Support Vector Machine:  0.872755577269


### Now with tf-idf scores... 

In [310]:
#reading in the data, this time in the form of paragraphs
caesar_p = gutenberg.paras('shakespeare-caesar.txt')
hamlet_p = gutenberg.paras('shakespeare-hamlet.txt')
macbeth_p = gutenberg.paras('shakespeare-macbeth.txt')

shakespear_p = caesar_p + hamlet_p + macbeth_p
# #processing
# shakespear_paras=[]
# for paragraph in shakespear_p:
#     para=paragraph[0]
#     #removing the double-dash from all words
#     para=[re.sub(r'--','',word) for word in para]
#     #Forming each paragraph into a string and adding it to the list of strings.
#     shakespear_paras.append(' '.join(para))
    
#processing
def process_paragraphs(paragraphs):
    _paras=[]
    for paragraph in paragraphs:
        for num in range(len(paragraph)):
            para=paragraph[num]
            #removing the double-dash from all words
            para=[re.sub(r'--','',word) for word in para]
            #Forming each paragraph into a string and adding it to the list of strings.
            _paras.append(' '.join(para))
    return _paras

shakespear_paras = process_paragraphs(shakespear_p)
print(shakespear_paras[0:10])
print(len(shakespear_paras))
print(len(caesar_p))
print(len(hamlet_p))
print(len(macbeth_p))

['[ The Tragedie of Julius Caesar by William Shakespeare 1599 ]', 'Actus Primus .', 'Scoena Prima .', 'Enter Flauius , Murellus , and certaine Commoners ouer the Stage .', 'Flauius .', 'Hence : home you idle Creatures , get you home : Is this a Holiday ?', 'What , know you not ( Being Mechanicall ) you ought not walke Vpon a labouring day , without the signe Of your Profession ?', 'Speake , what Trade art thou ?', 'Car .', 'Why Sir , a Carpenter']
7176
744
950
678


In [292]:
vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than half the paragraphs
                             min_df=2, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=True, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )


#Applying the vectorizer
shakespear_paras_tfidf=vectorizer.fit_transform(shakespear_paras)
print("Number of features: %d" % shakespear_paras_tfidf.get_shape()[1])
print(shakespear_paras_tfidf.get_shape())


#Reshapes the vectorizer output into something people can read
shakespear_paras_tfidf_csr = shakespear_paras_tfidf.tocsr()

#number of paragraphs
n = shakespear_paras_tfidf_csr.shape[0]
#A list of dictionaries, one per paragraph
tfidf_bypara = [{} for _ in range(0,n)]
#List of features
terms = vectorizer.get_feature_names()
#for each paragraph, lists the feature words and their tf-idf scores
# for i, j in zip(*shakespear_paras_tfidf_csr.nonzero()):
#     tfidf_bypara[i][terms[j]] = shakespear_paras_tfidf_csr[i, j]
    
#for each paragraph, lists the feature words and their tf-idf scores
for i, j in zip(*shakespear_paras_tfidf_csr.nonzero()):
    tfidf_bypara[0][terms[j]] = shakespear_paras_tfidf_csr[i, j]


Number of features: 3249
(7176, 3249)


In [293]:
# Each value is the count of the times the word appears in each sentence.
def ifidf_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += tfidf_bypara[0][word]
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

In [294]:
ifidf_df = ifidf_features(sentences, terms)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500


In [295]:
ifidf_df.head()

1599 abhorred abide aboord aboue abroad absence absent absolute absurd  \
0    0        0     0      0     0      0       0      0        0      0   
1    0        0     0      0     0      0       0      0        0      0   
2    0        0     0      0     0      0       0      0        0      0   
3    0        0     0      0     0      0       0      0        0      0   
4    0        0     0      0     0      0       0      0        0      0   

  abuse accent accents accesse accident accidentall according account  \
0     0      0       0       0        0           0         0       0   
1     0      0       0       0        0           0         0       0   
2     0      0       0       0        0           0         0       0   
3     0      0       0       0        0           0         0       0   
4     0      0       0       0        0           0         0       0   

  accounted accursed acquaint act acte acted acting action actions actor  \
0         0        0        0   0    0     0      0      0       0     0   
1         0        0        0   0    0     0      0      0       0     0   
2         0        0        0   0    0     0      0      0       0     0   
3         0        0        0   0    0     0      0      0       0     0   
4         0        0        0   0    0     0      0      0       0     0   

  actors acts actus adde added addition addrest adieu admiration admit  \
0      0    0     0    0     0        0       0     0          0     0   
1      0    0     0    0     0        0       0     0          0     0   
2      0    0     0    0     0        0       0     0          0     0   
3      0    0     0    0     0        0       0     0          0     0   
4      0    0     0    0     0        0       0     0          0     0   

  aduancement aduantage adue aduice aduise aeneas affaire affaires affayres  \
0           0         0    0      0      0      0       0        0        0   
1           0         0    0      0      0      0       0        0        0   
2           0         0    0      0      0      0       0        0        0   
3           0         0    0      0      0      0       0        0        0   
4           0         0    0      0      0      0       0        0        0   

  affear affection affections affliction affraid affrighted afraid afternoone  \
0      0         0          0          0       0          0      0          0   
1      0         0          0          0       0          0      0          0   
2      0         0          0          0       0          0      0          0   
3      0         0          0          0       0          0      0          0   
4      0         0          0          0       0          0      0          0   

  againe age agen ages ague ah aint ake alacke alarme alarum alarums alas  \
0      0   0    0    0    0  0    0   0      0      0      0       0    0   
1      0   0    0    0    0  0    0   0      0      0      0       0    0   
2      0   0    0    0    0  0    0   0      0      0      0       0    0   
3      0   0    0    0    0  0    0   0      0      0      0       0    0   
4      0   0    0    0    0  0    0   0      0      0      0       0    0   

  alexander aliue allegeance allow allowance allowed aloofe alreadie  \
0         0     0          0     0         0       0      0        0   
1         0     0          0     0         0       0      0        0   
2         0     0          0     0         0       0      0        0   
3         0     0          0     0         0       0      0        0   
4         0     0          0     0         0       0      0        0   

  altogether alwayes amaz amaze amazement ambassadors ambition ambitions  \
0          0       0    0     0         0           0        0         0   
1          0       0    0     0         0           0        0         0   
2          0       0    0     0         0           0        0         0   
3          0       0    0     0         0 

In [296]:
rfc2 = ensemble.RandomForestClassifier()
Y2 = ifidf_df['text_source']
X2 = np.array(ifidf_df.drop(['text_sentence','text_source'], 1))

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, 
                                                    Y2,
                                                    test_size=0.4,
                                                    random_state=0)
rfc2.fit(X2_train, y2_train)

print('Training set score:', rfc2.score(X2_train, y2_train))
print('\nTest set score:', rfc2.score(X2_test, y2_test))

Training set score: 0.971124097628

Test set score: 0.865979381443


In [297]:
scores_rfc2 = cross_val_score(rfc2, X2, Y2, cv=5)
scores_rfc2    

array([ 0.8753862 ,  0.85979381,  0.88762887,  0.87925697,  0.84520124])

In [298]:
lr2 = LogisticRegression(solver='newton-cg')

lr2.fit(X2_train, y2_train)

print(X2_train.shape, y2_train.shape)
print('Training set score:', lr2.score(X2_train, y2_train))
print('\nTest set score:', lr2.score(X2_test, y2_test))

(2909, 3249) (2909,)
Training set score: 0.865245788931

Test set score: 0.831443298969


In [299]:
scores_lr2 = cross_val_score(lr2, X2, Y2, cv=5)
scores_lr2

array([ 0.8146241 ,  0.80206186,  0.83917526,  0.84107327,  0.80185759])

In [300]:
clf2 = ensemble.GradientBoostingClassifier()

clf2.fit(X2_train, y2_train)

print('Training set score:', clf2.score(X2_train, y2_train))
print('\nTest set score:', clf2.score(X2_test, y2_test))

Training set score: 0.828119628738

Test set score: 0.812371134021


In [301]:
scores_clf2 = cross_val_score(clf2, X2, Y2, cv=5)
scores_clf2

array([ 0.81359423,  0.78350515,  0.83195876,  0.82972136,  0.78637771])

In [302]:
svm2 = SVC(kernel = 'linear')
svm2.fit(X2_train, y2_train)

print('Training set score:', svm2.score(X2_train, y2_train))
print('\nTest set score:', svm2.score(X2_test, y2_test))

Training set score: 0.880027500859

Test set score: 0.841237113402


In [303]:
scores_svm2 = cross_val_score(svm2, X2, Y2, cv=5)
scores_svm2

array([ 0.82183316,  0.81649485,  0.84123711,  0.85448916,  0.81114551])

In [305]:
print('Scores from BOW:')
print("Random Forest: ", np.mean(scores))
print("Logistic Regression: ", np.mean(scores2))
print("Gradient Boosting Classifier: ", np.mean(scores3))
print("Support Vector Machine: ", np.mean(scores4))


print(('\nScores from dfidf:'))
print("Random Forest: ", np.mean(scores_rfc2))
print("Logistic Regression: ", np.mean(scores_lr2))
print("Gradient Boosting Classifier: ", np.mean(scores_clf2))
print("Support Vector Machine: ", np.mean(scores_svm2))

Scores from BOW:
Random Forest:  0.842233289871
Logistic Regression:  0.861208328692
Gradient Boosting Classifier:  0.808206276465
Support Vector Machine:  0.872755577269

Scores from dfidf:
Random Forest:  0.869453416908
Logistic Regression:  0.819758413764
Gradient Boosting Classifier:  0.809031444297
Support Vector Machine:  0.829039959075


### Try it a little differently...

In [274]:
#reading in the data, this time in the form of paragraphs
caesar_p = gutenberg.paras('shakespeare-caesar.txt')
hamlet_p = gutenberg.paras('shakespeare-hamlet.txt')
macbeth_p = gutenberg.paras('shakespeare-macbeth.txt')


#processing
def process_paragraphs(paragraphs):
    _paras=[]
    for paragraph in paragraphs:
        for num in range(len(paragraph)):
            para=paragraph[num]
            #removing the double-dash from all words
            para=[re.sub(r'--','',word) for word in para]
            #Forming each paragraph into a string and adding it to the list of strings.
            _paras.append(' '.join(para))
    return _paras

In [311]:
caesar_paras = process_paragraphs(caesar_p)
print(caesar_paras[0:10])
print(len(caesar_paras))

['[ The Tragedie of Julius Caesar by William Shakespeare 1599 ]', 'Actus Primus .', 'Scoena Prima .', 'Enter Flauius , Murellus , and certaine Commoners ouer the Stage .', 'Flauius .', 'Hence : home you idle Creatures , get you home : Is this a Holiday ?', 'What , know you not ( Being Mechanicall ) you ought not walke Vpon a labouring day , without the signe Of your Profession ?', 'Speake , what Trade art thou ?', 'Car .', 'Why Sir , a Carpenter']
2163


In [312]:
hamlet_paras = process_paragraphs(hamlet_p)
print(hamlet_paras[0:10])
print(len(hamlet_paras))

['[ The Tragedie of Hamlet by William Shakespeare 1599 ]', 'Actus Primus .', 'Scoena Prima .', 'Enter Barnardo and Francisco two Centinels .', 'Barnardo .', "Who ' s there ?", 'Fran .', 'Nay answer me : Stand & vnfold your selfe', 'Bar .', 'Long liue the King']
3106


In [313]:
macbeth_paras = process_paragraphs(macbeth_p)
print(macbeth_paras[0:10])
print(len(macbeth_paras))

['[ The Tragedie of Macbeth by William Shakespeare 1603 ]', 'Actus Primus .', 'Scoena Prima .', 'Thunder and Lightning .', 'Enter three Witches .', '1 .', 'When shall we three meet againe ?', 'In Thunder , Lightning , or in Raine ?', '2 .', "When the Hurley - burley ' s done , When the Battaile ' s lost , and wonne"]
1907


In [306]:
#Our SVD data reducer.  We are going to reduce the feature space from 1379 to 130.
svd= TruncatedSVD(130)
lsa = make_pipeline(svd, Normalizer(copy=False))
# Run SVD on the training data, then project the training data.
shakespear_paras_tfidf_lsa = lsa.fit_transform(shakespear_paras_tfidf)

variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100)

#Looking at what sorts of paragraphs our solution considers similar, for the first five identified topics
paras_by_component=pd.DataFrame(shakespear_paras_tfidf_lsa,index=shakespear_paras)
for i in range(5):
    print('Component {}:'.format(i))
    print(paras_by_component.loc[:,i].sort_values(ascending=False)[0:10])




Percent variance captured by all components: 49.4319604581
Component 0:
Ham .    1.0
Ham .    1.0
Ham .    1.0
Ham .    1.0
Ham .    1.0
Ham .    1.0
Ham .    1.0
Ham .    1.0
Ham .    1.0
Ham .    1.0
Name: 0, dtype: float64
Component 1:
Bru .    1.0
Bru .    1.0
Bru .    1.0
Bru .    1.0
Bru .    1.0
Bru .    1.0
Bru .    1.0
Bru .    1.0
Bru .    1.0
Bru .    1.0
Name: 1, dtype: float64
Component 2:
Macb .    1.0
Macb .    1.0
Macb .    1.0
Macb .    1.0
Macb .    1.0
Macb .    1.0
Macb .    1.0
Macb .    1.0
Macb .    1.0
Macb .    1.0
Name: 2, dtype: float64
Component 3:
This is one Lucianus nephew to the King    0.997475
King .                                     0.997046
King .                                     0.997046
King .                                     0.997046
King .                                     0.997046
King .                                     0.997046
King .                                     0.997046
King .                                     0.997046
K

In [349]:
paras_by_component.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129
[ The Tragedie of Julius Caesar by William Shakespeare 1599 ],-7.312483e-11,6.731321e-06,3.170721e-06,3.140696e-03,7.252969e-05,1.083663e-02,1.252102e-05,-1.212356e-07,0.019133,1.083587e-04,0.000021,3.532008e-04,9.963124e-05,0.000849,0.002021,0.186410,-9.128780e-06,-0.009715,-8.199788e-04,0.000004,-0.000054,0.000765,0.048602,0.030907,0.000072,0.290241,0.000017,4.819319e-05,-0.204764,0.030992,0.000035,0.062031,0.466903,0.000253,-0.176660,0.069672,0.000271,-0.027817,-0.457007,-0.471493,0.233879,0.000160,0.018405,0.021425,0.000077,-0.006853,-0.133461,0.000050,0.037947,0.001125,-0.069065,-0.044320,-0.022364,-0.000785,-0.004246,-0.032803,0.001625,0.013626,0.022203,0.081893,0.076873,0.007160,0.026197,0.002722,-0.013876,-0.008637,0.006521,-0.026614,0.029055,-0.008283,0.005989,-0.005859,-0.014955,0.098569,-0.000859,0.018391,-0.032910,0.032934,0.000927,0.030330,0.012423,-0.001845,0.070553,0.017311,-0.003490,0.060577,-0.007153,0.019925,-0.002169,0.032389,-0.036995,0.017454,-0.032994,-0.013621,-0.005118,-0.071700,-0.031519,0.034133,-0.004985,-0.013767,0.000708,0.024245,-0.014144,0.007061,0.002011,-0.022349,-0.019431,-0.021617,-0.005803,0.032159,0.019950,-0.001988,0.003722,0.007702,-0.002875,0.009994,-0.005253,0.029943,-0.003416,-0.008863,0.008852,-0.013349,0.003155,0.004528,0.006585,-0.024013,0.016451,-0.034692,0.032516,-0.049333
Actus Primus .,-6.824578e-10,-3.698926e-08,1.345923e-07,2.899573e-07,-8.616948e-07,3.531881e-06,2.844427e-07,-8.624027e-06,-0.000005,1.274346e-05,0.000014,-7.327647e-06,3.028241e-05,-0.000054,-0.000069,-0.000038,-2.679186e-04,0.000013,-3.402427e-04,-0.000111,0.000197,0.000384,0.000429,-0.000497,-0.000407,-0.001178,0.001814,-8.181869e-04,0.000453,0.001573,-0.000240,0.000255,0.000792,-0.001969,0.001657,-0.001255,-0.001216,0.003298,-0.001630,-0.002337,-0.005315,-0.000178,0.002261,0.002111,-0.010253,0.008800,-0.007796,-0.005086,-0.005993,-0.005689,-0.007901,-0.019723,-0.024122,-0.009000,0.001820,0.010588,-0.035228,-0.027479,0.020528,0.014579,0.024828,0.008759,-0.014194,0.001811,0.003047,-0.037750,-0.031830,0.015822,0.006489,-0.018583,0.014091,-0.055384,0.023751,-0.031609,-0.022117,0.020725,-0.005604,0.010109,-0.073358,-0.032031,0.058809,-0.008262,-0.115025,0.075231,-0.008621,-0.029854,-0.030412,-0.029016,0.000381,-0.022383,0.019643,0.063760,0.054054,0.050976,0.094080,0.047945,0.124435,-0.014988,-0.023556,0.024637,0.008149,-0.151272,0.099698,-0.139580,0.023940,-0.002563,-0.168971,-0.049799,0.081112,-0.069861,-0.172637,-0.157446,-0.172320,-0.094407,-0.039976,0.013891,0.092987,-0.202915,0.285130,-0.075144,-0.036933,-0.132205,-0.050994,0.022813,0.696763,0.231091,0.090088,-0.040501,0.080979,0.107021
Scoena Prima .,3.994165e-10,-9.197317e-09,5.388548e-08,-3.134900e-08,-8.813210e-08,3.428989e-07,-1.915759e-07,-1.063763e-06,-0.000002,6.905867e-08,-0.000002,-1.941207e-07,-8.108227e-07,0.000004,-0.000003,0.000010,9.181287e-06,-0.000008,-9.480388e-07,0.000036,-0.000061,-0.000051,0.000078,0.000049,0.000068,-0.000206,-0.000113,-2.589745e-04,0.000019,-0.000081,0.000249,0.000574,-0.000293,-0.000369,-0.000622,0.000528,-0.000211,-0.000024,0.000742,-0.000299,-0.000386,0.000261,-0.000335,-0.000933,0.000420,-0.000759,-0.000079,0.000879,-0.000053,-0.001030,-0.000773,0.000112,0.000192,0.001311,0.003321,-0.004358,0.003467,-0.000171,0.005171,-0.003644,-0.001262,0.003719,0.001612,0.003141,-0.000943,0.007608,0.007125,-0.007133,-0.004113,-0.003902,0.002523,0.005674,-0.000361,0.000192,0.002544,-0.007788,0.034213,0.012639,0.026966,0.023681,0.985423,-0.032273,-0.051856,0.003916,0.006917,-0.010989,-0.000176,0.018684,0.005989,-0.011765,-0.00537

In [314]:
caesar_length =(len(caesar_paras))
hamlet_length = (len(hamlet_paras))
macbeth_length = (len(macbeth_paras))

In [317]:
text_source = []
for num in range(0, len(caesar_paras)):
    text_source.append('Caesar')
for num in range(0, len(hamlet_paras)):
    text_source.append('Hamlet')
for num in range(0, len(macbeth_paras)):
    text_source.append('Macbeth')
print(len(text_source))

7176


In [319]:
rfc3 = ensemble.RandomForestClassifier()
Y3 = text_source
X3 = np.array(paras_by_component)

X3_train, X3_test, y3_train, y3_test = train_test_split(X3, 
                                                    Y3,
                                                    test_size=0.4,
                                                    random_state=0)
rfc3.fit(X3_train, y3_train)

print('Training set score:', rfc3.score(X3_train, y3_train))
print('\nTest set score:', rfc3.score(X3_test, y3_test))

Training set score: 0.938211382114

Test set score: 0.667711598746


In [320]:
scores_rfc3 = cross_val_score(rfc3, X3, Y3, cv=5)
scores_rfc3   

array([ 0.63535143,  0.66016713,  0.62020906,  0.65969317,  0.62831241])

In [323]:
lr3 = LogisticRegression(solver='newton-cg')

lr3.fit(X3_train, y3_train)

# print(X3_train.shape, y3_train.shape)
print('Training set score:', lr3.score(X3_train, y3_train))
print('\nTest set score:', lr3.score(X3_test, y3_test))

Training set score: 0.724738675958

Test set score: 0.699407871822


In [324]:
scores_lr3 = cross_val_score(lr3, X3, Y3, cv=5)
scores_lr3 

array([ 0.66388309,  0.68454039,  0.66620209,  0.69386332,  0.66317992])

In [325]:
clf3 = ensemble.GradientBoostingClassifier()

clf3.fit(X3_train, y3_train)

print('Training set score:', clf3.score(X3_train, y3_train))
print('\nTest set score:', clf3.score(X3_test, y3_test))

Training set score: 0.815098722416

Test set score: 0.711598746082


In [326]:
scores_clf3 = cross_val_score(clf3, X3, Y3, cv=5)
scores_clf3 

array([ 0.66945024,  0.68732591,  0.67317073,  0.68061367,  0.67294282])

In [327]:
svm3 = SVC(kernel = 'linear')
svm3.fit(X3_train, y3_train)

print('Training set score:', svm3.score(X3_train, y3_train))
print('\nTest set score:', svm3.score(X3_test, y3_test))

Training set score: 0.720557491289

Test set score: 0.695576454197


In [328]:
scores_svm3 = cross_val_score(svm3, X3, Y3, cv=5)
scores_svm3 

array([ 0.64231037,  0.67688022,  0.67038328,  0.67712692,  0.62622036])

In [329]:
print('Scores from BOW:')
print("Random Forest: ", np.mean(scores))
print("Logistic Regression: ", np.mean(scores2))
print("Gradient Boosting Classifier: ", np.mean(scores3))
print("Support Vector Machine: ", np.mean(scores4))


print(('\nScores from dfidf:'))
print("Random Forest: ", np.mean(scores_rfc2))
print("Logistic Regression: ", np.mean(scores_lr2))
print("Gradient Boosting Classifier: ", np.mean(scores_clf2))
print("Support Vector Machine: ", np.mean(scores_svm2))

print(('\nScores from dfidf with reduction:'))
print("Random Forest: ", np.mean(scores_rfc3))
print("Logistic Regression: ", np.mean(scores_lr3))
print("Gradient Boosting Classifier: ", np.mean(scores_clf3))
print("Support Vector Machine: ", np.mean(scores_svm3))

Scores from BOW:
Random Forest:  0.842233289871
Logistic Regression:  0.861208328692
Gradient Boosting Classifier:  0.808206276465
Support Vector Machine:  0.872755577269

Scores from dfidf:
Random Forest:  0.869453416908
Logistic Regression:  0.819758413764
Gradient Boosting Classifier:  0.809031444297
Support Vector Machine:  0.829039959075

Scores from dfidf with reduction:
Random Forest:  0.640746639107
Logistic Regression:  0.674333761208
Gradient Boosting Classifier:  0.676700673184
Support Vector Machine:  0.658584229452


In [337]:
tfidf_df2 = pd.SparseDataFrame([ pd.SparseSeries(shakespear_paras_tfidf[i].toarray().ravel()) 
                              for i in np.arange(shakespear_paras_tfidf.shape[0]) ])

In [339]:
tfidf_df2.shape

(7176, 3249)

In [341]:
rfc4 = ensemble.RandomForestClassifier()
Y4 = text_source
X4 = np.array(tfidf_df2)

X4_train, X4_test, y4_train, y4_test = train_test_split(X4, 
                                                    Y4,
                                                    test_size=0.4,
                                                    random_state=0)
rfc4.fit(X4_train, y4_train)

print('Training set score:', rfc4.score(X4_train, y4_train))
print('\nTest set score:', rfc4.score(X4_test, y4_test))

Training set score: 0.936353077816

Test set score: 0.723789620341


In [345]:
scores_rfc4 = cross_val_score(rfc4, X4, Y4, cv=5)
scores_rfc4 

array([ 0.65901183,  0.70194986,  0.65644599,  0.66108787,  0.60041841])

In [342]:
lr4 = LogisticRegression(solver='newton-cg')

lr4.fit(X4_train, y4_train)

# print(X3_train.shape, y3_train.shape)
print('Training set score:', lr4.score(X4_train, y4_train))
print('\nTest set score:', lr4.score(X4_test, y4_test))

Training set score: 0.872473867596

Test set score: 0.740856844305


In [346]:
scores_lr4 = cross_val_score(lr4, X4, Y4, cv=5)
scores_lr4

array([ 0.69032707,  0.70473538,  0.69059233,  0.69735007,  0.67224547])

In [343]:
clf4 = ensemble.GradientBoostingClassifier()

clf4.fit(X4_train, y4_train)

print('Training set score:', clf4.score(X4_train, y4_train))
print('\nTest set score:', clf4.score(X4_test, y4_test))

Training set score: 0.705923344948

Test set score: 0.681992337165


In [344]:
svm4 = SVC(kernel = 'linear')
svm4.fit(X4_train, y4_train)

print('Training set score:', svm4.score(X4_train, y4_train))
print('\nTest set score:', svm4.score(X4_test, y4_test))

Training set score: 0.89012775842

Test set score: 0.737722048067


In [ ]:
scores_svm4 = cross_val_score(svm4, X4, Y4, cv=5)
scores_svm4